In [1]:
import numpy as np
np.random.seed(7)

import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)
from keras.engine.topology import Layer
from keras.layers import Dense, Dropout
from keras.losses import mean_squared_error

Using TensorFlow backend.


In [2]:
from retreive_data import *

print "unpacking data"
x_data = list()
y_data = list()
for i in ["1", "2"]: #, "2", "3", "4", #"5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21"]:
    x, y = unpack_gzip("chr" + i + ".txt.gz", func = no_change, ragged = True)
    x_data.extend(list(x))
    y_data.extend(list(y))

n = len(x_data)
try:
    assert (n == len(y_data))
except AssertionError:
    print str(n) + " and " + str(len(y_data)) + " do not match"

#Shuffling
d = zip(x_data, y_data)
np.random.shuffle(d)
x_data, y_data = zip(*d)
x_data, y_data = np.asarray(x_data), np.asarray(y_data)

#Taking log of RNA/DNA
y_data=np.log(y_data)

#4/6 as training data, 1/6 as test, 1/6 as validation                                                                
x_dim, y_dim = (4, 500)                                                                        
train = (x_data[2*n/6:], y_data[2*n/6:])                                                                  
test = (x_data[:n/6], y_data[:n/6])
val = (x_data[n/6:2*n/6], y_data[n/6:2*n/6])

print "done unpacking data"

unpacking data


ValueError: could not broadcast input array from shape (4,251) into shape (4)

In [8]:
print x_data.shape
print y_data.shape

x_data = np.asarray(x_data, dtype=object)
y_data = np.asarray(y_data, dtype=object)

train[0] = train[0].reshape(train[0].shape[0], x, y, 1)
test[0] = test[0].reshape(test[0].shape[0], x, y, 1)
va[0] = val[0].reshape(val[0].shape[0], x, y, 1)
print "training shape: " + str(train_data.shape)

(511479, 4, 500)
(511479,)


TypeError: only integer scalar arrays can be converted to a scalar index

In [25]:
#Convolution layer that takes in variable size input

epochs = 12
batch_size = 5000

with tf.name_scope("input_vars"):
    x = tf.placeholder(tf.int32, shape=[None, 4, None, 1])
    y = tf.placeholder(tf.float32, shape=[None, 1])

# #Define model
# x = tf.nn.conv2d(input=train_data,
#              filter=128,
#              strides=[1, 4, 1, 1],
#              padding="VALID")

    x = Dense(10, activation='relu')(x)
    x = Dropout(.2)(x)
    results = Dense(1)(x)

#Define training
loss = tf.reduce_mean(mean_squared_error(train_labels, results))

train_step = tf.train.AdamOptimizer().minimize(loss)

TypeError: Input 'b' of 'MatMul' Op has type float32 that does not match type int32 of argument 'a'.

In [14]:
#Training model

#Init variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

with sess.as_default():
    for i in range(x_data.shape[0]/batch_size):
        batch = x_data[i*batch_size:(i+1)*batch_size]
        train_step.run(feed_dict={train_data: train[0],
                                 train_labels: train[1]})

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.

In [ ]:
#Evaluate model
acc = tf.reduce_mean(mean_squared_error(train_labels, results)) #same as loss above
with sess.as_default():
    print acc.eval(feed_dict={train_data: x_data,
                             train_labels: y_data})